<a href="https://colab.research.google.com/github/danielbryksil/Calculadora-LF-Secund-rio/blob/main/Calculadora_de_Carteira_LF_CDI_%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Calculadora PU CDI +


In [1]:
!pip install python-bcb
!pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 4.4 MB/s eta 0:00:

In [2]:
from datetime import datetime
import pandas as pd
from bcb import sgs
import numpy as np
from datetime import datetime, timedelta
from google.colab import files
import pandas as pd
import requests

In [3]:
dados_excel = pd.read_excel('/content/feriados.xlsx')
feriados = dados_excel['2001-01-01'].tolist()

FileNotFoundError: [Errno 2] No such file or directory: '/content/feriados.xlsx'

In [ ]:
df = pd.read_excel('/content/lf secun python cdi +.xlsx')
df

In [ ]:
data_operação = input("Qual a data da operação (aaaa-mm-dd)? ")

In [ ]:
data_str = data_operação
data = datetime.strptime(data_str, '%Y-%m-%d')
data_menos_um_dia = data - timedelta(days=1)

# Converta o resultado de volta para o formato desejado
data_menos_um_dia_operação = data_menos_um_dia.strftime('%Y-%m-%d')
data_menos_um_dia_operação

In [ ]:
data1 = pd.to_datetime(df['Data Vencimento'])
data2 = pd.to_datetime(data_operação)
df['dias_venc'] = (data1 - data2)
df['dias_venc'] = (df['dias_venc'].dt.days)-1


In [ ]:
data_cdi = sgs.get(('CDI', 12), last=3000)
data_cdi = data_cdi['CDI']
data_cdi

In [ ]:
di_per_list = []

for a, b in zip(df['Data Emissão'], df['Taxa de Emissão']):
    data_filt = data_cdi.loc[a:data_menos_um_dia_operação]
    di = ((data_filt/100)+1)*((1+b)**(1/252))
    di_per = di.product()
    di_per_list.append(di_per)

df['di_acum'] = di_per_list

In [ ]:
# URL da página
url = 'https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/TxRef1.asp'

# Realiza a solicitação HTTP
response = requests.get(url)

# Verifica se a solicitação foi bem-sucedida (código de status 200)
if response.status_code == 200:
    # Lê a tabela HTML diretamente usando o pandas
    dfs = pd.read_html(response.text, flavor='bs4')  # 'bs4' indica o uso do BeautifulSoup para análise HTML

    # O resultado será uma lista de DataFrames, onde cada DataFrame representa uma tabela na página
    # Dependendo da página, você pode precisar procurar qual DataFrame contém os dados desejados
    for i, b in enumerate(dfs):
        print(f"DataFrame {i + 1}:\n{b}")
else:
    print(f"Falha na solicitação. Código de status: {response.status_code}")

di_tabela = dfs[2]
di_tabela = di_tabela.set_index(0)
di_tabela = di_tabela.iloc[2:]
di_tabela = di_tabela[1]
di_tabela = di_tabela.astype(float)
di_tabela.index = di_tabela.index.astype(int)
di_tabela = di_tabela/100


In [ ]:
df_di_tabela = pd.DataFrame(di_tabela)


In [ ]:
def interpolar_valor(df, x):
  # Encontrar a linha na tabela onde o valor X está presente ou próximo.
  linha = df.index[df.index <= x].max()

  # Se o valor X já estiver presente na tabela DI, retornar o valor correspondente.
  if linha == x:
    return round(df.loc[linha,1],2)

  # Se o valor X não estiver presente na tabela DI, interpolar.
  if linha != x:
    di_anterior = df_di_tabela.loc[linha,1]
    posicao = df_di_tabela.index.get_loc(linha)
    # Obter o próximo índice
    proximo_indice = posicao + 1
    di_prox = df_di_tabela[1].iloc[proximo_indice]
    prox_num_dias_di = df_di_tabela.index[proximo_indice]
    interpolação = di_anterior + (x-linha) * ((di_prox-di_anterior) / (prox_num_dias_di-linha))
    return round(interpolação,2)

  return 0

In [ ]:
di_dia_list = []

for a in df['dias_venc']:
  tx_di = interpolar_valor(df_di_tabela, a) /100
  di_dia_list.append(tx_di)

df['di_dia'] = di_dia_list
df['pu_par'] = df['PU Inicial'] * df['di_acum']


In [ ]:
num_dias_uteis = []

for a in df['Data Emissão']:
    datas = pd.date_range(start=a, end=data_operação, freq='B')
    dias_uteis = datas[~datas.isin(feriados)]
    dias_uteis_b = len(dias_uteis)-1
    num_dias_uteis.append(dias_uteis_b)


df['num_dias_uteis'] = num_dias_uteis
num_dias_uteis_oper_venc = []

for a in df['Data Vencimento']:
    datas = pd.date_range(start=data_operação, end=a, freq='B')
    dias_uteis = datas[~datas.isin(feriados)]
    dias_uteis_b = len(dias_uteis)-1
    num_dias_uteis_oper_venc.append(dias_uteis_b)


df['num_dias_uteis_oper'] = num_dias_uteis_oper_venc


In [ ]:
df['di_int_acum'] =((((1+df['di_dia'])**(1/252))*((1+df['Taxa de Emissão'])**(1/252)))**(252))-1
df['di_int_acum_2'] = +(1+df['di_int_acum'])**(df['num_dias_uteis_oper']/252)-1
df['neg_di_acum']=((((1+df['Taxa de Venda'])**(1/252))*((1+df['di_dia'])**(1/252)))**(252))-1
df['neg_di_acum_2'] =+(1+df['neg_di_acum'])**(df['num_dias_uteis_oper']/252)-1
df['vf'] = (df['di_int_acum_2']+1) * df['pu_par']
df['pu_final'] = df['vf'] / (1+ df['neg_di_acum_2'])

In [ ]:
df_filtro = df[['Tipo Aplic.', 'Data Inicial', 'Emissor', 'Banco',
       'Valor Inicial', 'Taxa COMPRA', 'Transação', 'Titulo',
       'Data Emissão', 'Data Vencimento', 'PU Inicial', 'Taxa de Emissão', 'Taxa de Venda','pu_final','Quantidade']]

In [ ]:
df_filtro

In [ ]:
carteira = pd.read_excel('/content/Carteira.xlsm')
carteira

In [ ]:
resultado = pd.merge(df_filtro, carteira[['Transação','Valor Aplicado']], on='Transação', how='left')
resultado['Valor Mercado'] = resultado['pu_final'] * resultado['Quantidade']
resultado['Ganho/Perda'] = resultado['Valor Mercado'] - resultado['Valor Aplicado']
resultado['Ganho/Perda %'] = resultado['Ganho/Perda'] / resultado['Valor Aplicado']
resultado

In [ ]:
nome_arquivo_excel2 = 'LF_cdi+.xlsx'
resultado.to_excel(nome_arquivo_excel2, index=False)

In [ ]:
# Fazer o download do arquivo Excel
files.download(nome_arquivo_excel2)